In [1]:
import tensorflow as tf
from keras.src.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import os
import random
from PIL import Image
import numpy as np
from skimage import filters
from sklearn.model_selection import train_test_split
from datetime import datetime

In [2]:
def create_cnn(input_shape, num_classes1, num_classes2):
    input_layer = layers.Input(shape=input_shape)

    conv1 = tf.keras.layers.Conv2D(48, (11, 11), activation='relu', strides=4, padding='same')(input_layer)
    batch1 = tf.keras.layers.BatchNormalization()(conv1)

    pool1 = tf.keras.layers.MaxPooling2D((3, 3), strides=2)(batch1)

    conv2 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(pool1)
    batch3 = tf.keras.layers.BatchNormalization()(conv2)

    pool2 = tf.keras.layers.MaxPooling2D((3, 3), strides=2)(batch3)

    conv3 = tf.keras.layers.Conv2D(384, (3, 3), activation='relu', strides=1, padding='same')(pool2)
    batch5 = tf.keras.layers.BatchNormalization()(conv3)

    conv4 = tf.keras.layers.Conv2D(384, (3, 3), activation='relu', strides=1, padding='same')(batch5)
    batch6 = tf.keras.layers.BatchNormalization()(conv4)

    conv5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(batch6)
    batch7 = tf.keras.layers.BatchNormalization()(conv5)

    pool3 = tf.keras.layers.MaxPooling2D((3, 3), strides=2, padding='same')(batch7)

    flatten = tf.keras.layers.Flatten()(pool3)

    l2 = tf.keras.regularizers.L2(l2=1e-4)

    dense1 = layers.Dense(2048, activation='relu')(flatten)
    batch9 = tf.keras.layers.BatchNormalization()(dense1)
    dense2 = layers.Dense(2048, activation='relu')(batch9)
    batch10 = tf.keras.layers.BatchNormalization()(dense2)

    output1 = tf.keras.layers.Dense(num_classes1, activation='softmax', kernel_regularizer=l2, name='output1')(batch10)
    output2 = tf.keras.layers.Dense(num_classes2, activation='sigmoid', kernel_regularizer=l2, name='output2')(batch10)

    label_layer_1 = tf.keras.layers.Input((num_classes1,))
    label_layer_2 = tf.keras.layers.Input((1,))

    model = tf.keras.Model(inputs=[input_layer, label_layer_1, label_layer_2], outputs=[output1, output2])

    lambda_const = tf.constant(0.5)

    categorical_loss = tf.reduce_mean(tf.math.negative(tf.math.multiply(tf.math.subtract(tf.constant(1.0), lambda_const), (tf.reduce_sum(tf.math.multiply(label_layer_1, tf.math.log(output1)), axis=1)))))

    binary_loss = tf.math.multiply(lambda_const, tf.reduce_mean(tf.math.subtract(tf.math.negative(tf.math.multiply(label_layer_2, tf.math.log(output2))), tf.math.multiply(tf.math.subtract(tf.constant(1.0), label_layer_2), tf.math.log(tf.math.subtract(tf.constant(1.0), output2))))))

    loss = categorical_loss + binary_loss

    model.add_loss(loss)

    return model

In [3]:
model = create_cnn((220, 150, 1), 531, 2)

In [7]:
model.layers[1]

In [8]:
model.layers[1].output_shape

(None, 55, 38, 96)

In [194]:
model.layers